In [12]:
import import_ipynb
import operator
from config import BASE_DIR
import pickle
import numpy as np
from extract_info import cid2vect
#from clustering2 import *

In [28]:
def map_medoids():
    try:
        # load mapping
        infile = open(BASE_DIR+'/medoid_cluster_dict.pkl', 'rb')
        mapping = pickle.load(infile)
        infile.close()
        return mapping 
    except IOError:
        print("File Not Found or wrong path")
    
    
# creating bounding box for each dimentions(particularly in our case its 300 dims) 
# of every compound in the cluster 
def get_bbox(vec):
    bbox = LB_N_UB(vec)
    box = []
    for index in range(0, 300):
        temp = [bbox[0][0][index], bbox[1][0][index]]
        box.append(temp)
    return box

# computing variance
# returns dictionary [key -> clusterID: value -> [Bounding Box, MAD value]]
def compute_variance(medoids):
    
    #keys = list(medoids.keys()) 
    mad_dict = dict()
    clusters = list(medoids.items())

    for cluster_list in clusters:
        med = cluster_list[1][0] # check whether medoid is None
        if med is not None:
            medoid = cluster_list[1][0]
            medoid_vec = cid2vect(medoid)[0] # medoid vector
            
            # extracting vectors for each compounds from the cluster
            cluster_vec = [cid2vect(x) for x in cluster_list[1][1]] 
            cluster_bbox = get_bbox(cluster_vec) # extracting bounding box of current cluster
            cluster = [x for vec in cluster_vec for x in vec]
            key = cluster_list[0]
            mad_value = MAD(cluster, medoid_vec) # computing variance for current cluster
            temp = {str(key): [cluster_bbox, mad_value]} # [clusterID, Bounding Box, MAD]
            mad_dict.update(temp)
        else:
            parent_cluster = cluster_list[1][2]
            temp = {cluster_list[0] :mad_dict.get(parent_cluster)}
            mad_dict.update(temp)
    print(len(mad_dict)) 
    return mad_dict

#where vec is each value in sample space cluster and medoid is an average of sample sapce
#reference for computation of MAD : https://en.wikipedia.org/wiki/Median_absolute_deviation
def get_median_absolute_deviation(cluster, medoid):
    differences = []
    for vec in cluster:
        difference = np.abs(list(map(operator.sub, vec, medoid))) # squared each value or observation
        difference.sort()
        differences.append(list(difference))
    
    absolute_differences = np.array(differences)
    mad = np.sqrt(np.median(absolute_differences)) 

    return mad


# computing variance
MAD = lambda X, median :  get_median_absolute_deviation(X, median)


# generate lower bound and upper bound
LB_N_UB = lambda vec : [np.min(vec, axis=0), np.max(vec, axis=0)]


# creating dictionary where each cluster have bounding box and MAD value
def store_mad_dict(mad_dict):
    filename = 'variance_dict.pkl'
    outfile = open(filename,'wb+')
    pickle.dump(mad_dict,outfile)
    outfile.close 
    
# load variance dictionary    
def map_variance():
    try:
        # load mapping
        infile = open(BASE_DIR+'/variance_dict.pkl', 'rb')
        mapping = pickle.load(infile)
        infile.close()
        return mapping 
    except IOError:
        print("File Not Found or wrong path")    

In [26]:
# mapping computed medoids (require for the computation of variance)
medoid_list = map_medoids()
#medoid_list

mad_dict = compute_variance(medoid_list)
store_mad_dict(mad_dict)

37


In [27]:

var = map_variance()
print(var)

{'C0': [[[-2.3197932, 4.2513165], [-4.731459, 2.6177986], [-4.827512, 0.1388317], [-3.7869797, 4.5140877], [-3.6724439, 6.3012347], [-8.189604, 2.01474], [-15.719692, -1.050158], [-2.3722813, 4.525275], [-2.4823034, 4.3687925], [-4.2175703, 4.1906185], [-2.6923711, 8.473705], [-1.2249974, 3.3446503], [-7.050714, 8.712341], [-1.9461291, 7.475397], [-12.5405855, 0.2702784], [-6.685646, 3.8443887], [-2.6691332, 6.548427], [-11.333618, -0.14045413], [-9.421626, -0.4793135], [1.0734053, 9.273585], [-0.020897701, 6.3353415], [-0.3894165, 5.9958825], [2.488234, 19.225891], [1.1301523, 11.039065], [-9.359862, -0.5855175], [-2.2326014, 2.8914587], [-11.174367, 1.1010759], [-9.59745, 1.0606791], [-2.6549683, 6.9362183], [-6.3243327, 0.54331464], [-1.1525608, 8.244811], [-5.9440937, 0.7327907], [-7.094335, 0.7394067], [-4.5221004, 5.057487], [-2.7342112, 3.9224536], [-2.6004412, 4.0197043], [-5.659411, 1.2396551], [-2.8542132, 2.9115658], [1.5618052, 13.628518], [-4.007164, 7.4104013], [-4.618738